In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np

from dataset import HealthcareDataset
from model import initialize_model

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# Load trained model
net = initialize_model()

SAVED_MODEL_PATH = 'checkpoints/mobilenet_1'
net.load_state_dict(torch.load(SAVED_MODEL_PATH))

# Freeze layers
for param in net.parameters():
    param.requires_grad = False
    
# Now replace the classifier with an 11-output net to predict
# the 11 healthcare outcomes
net.classifier = nn.Linear(1280, 11)
for param in net.classifier.parameters():
    param.requires_grad = True
    
net.to(device)

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(9, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=Tr

In [4]:
# Get dataset
DATA_FOLDER = 'E:/xplore_data/data/'
HEALTH_FILE = 'data/dhs_gps.csv'
dtrain = HealthcareDataset(DATA_FOLDER, HEALTH_FILE)
[x.shape for x in dtrain.__getitem__(0)]

[torch.Size([9, 333, 333]), torch.Size([]), torch.Size([11])]

In [8]:
def train_model(model, dataloader, optimizer, scheduler, num_epochs=4):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        model.train()
        running_loss = 0.0
        
        criterion = nn.MSELoss()

        # Iterate over data.
        for x, _, y in dataloader:
            x = x.to(device)
            y = y.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                outputs = model(x)
                loss = criterion(outputs, y)
                loss.backward()
                optimizer.step()
                scheduler.step()

            # statistics
            running_loss += loss.item() * x.size(0)

        epoch_loss = running_loss / len(dtrain)

        print('Loss: {:.4f}'.format(
            epoch_loss))
    return model


In [9]:
BATCH_SIZE = 32
EPOCHS = 100

# Data loader
dloader = torch.utils.data.DataLoader(dtrain, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
# Create training optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)
# Optimizer LR decay
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)


In [10]:
net = train_model(net, dloader, optimizer, scheduler, EPOCHS)

Epoch 0/99
----------
Loss: 0.1320
Epoch 1/99
----------
Loss: 0.0953
Epoch 2/99
----------
Loss: 0.0725
Epoch 3/99
----------


KeyboardInterrupt: 